# MNIST 

In [ ]:
#!pip install jovian --upgrade --quiet
#import jovian
#jovian.commit(project='MNIST_Pytorch', environment=None)

In [ ]:
# Numerical computation and visualization libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Pytorch Libraries
import torch
import torchvision
import torchvision.transforms as tr
import torch.nn as nn
from torch.utils.data import random_split
import torch.optim as optim
from torchvision import models
import copy

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Dataset 

trainset = torchvision.datasets.MNIST(root='./data',
                                      train=True,
                                      download=True,
                                      #transform=tr.ToTensor()
                                      )


testset = torchvision.datasets.MNIST(root='./data',
                                     train=False,
                                     download=True,
                                     #transform=tr.ToTensor()
                                    )

print('Len of train dataset ',len(trainset))
print('Len of test dataset ',len(testset))
print('-'*30)
print(trainset)
print(testset)
print('-'*30)
print(trainset[0])
print(testset[0])
print('-'*30)
trainset[0][0]

In [ ]:
# Loading images and labels for fifth item dataitem

image , label = trainset[4]
plt.imshow(image,cmap='gray')
print('Label :',label)

In [ ]:
fig , ax = plt.subplots(nrows=2,ncols=3,figsize=(10,10))

image , label = trainset[0]
ax[0,0].imshow(image)
ax[0,0].set_title(label)
ax[0,0].axis('off')

image , label = trainset[1]
ax[0,1].imshow(image)
ax[0,1].set_title(label)
ax[0,1].axis('off')

image , label = trainset[2]
ax[0,2].imshow(image)
ax[0,2].set_title(label)
ax[0,2].axis('off')

image , label = trainset[3]
ax[1,0].imshow(image)
ax[1,0].set_title(label)
ax[1,0].axis('off')

image , label = trainset[4]
ax[1,1].imshow(image)
ax[1,1].set_title(label)
ax[1,1].axis('off')

image , label = trainset[5]
ax[1,2].imshow(image)
ax[1,2].set_title(label)
ax[1,2].axis('off')

plt.tight_layout(pad=2)

In [ ]:
#Converting the Dataset into tensors

trainset = torchvision.datasets.MNIST(root='./data',
                                      train=True,
                                      download=True,
                                      transform=tr.ToTensor()
                                      )


testset = torchvision.datasets.MNIST(root='./data',
                                     train=False,
                                     download=True,
                                     transform=tr.ToTensor()
                                    )

In [ ]:
# Split the data into train and validation set

trainset , valset = random_split(trainset,[50000,10000])
len(trainset),len(valset)

In [ ]:
batch_size= 256

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True)

valloader = torch.utils.data.DataLoader(valset,
                                          batch_size=batch_size,
                                          shuffle=False)

testloader = torch.utils.data.DataLoader(testset,
                                          batch_size=batch_size,
                                          shuffle=False)

classes = ('0','1','2','3','4','5','6','7','8','9')

In [ ]:
images , labels= next(iter(trainloader))
print(images.shape)
print(images[1].shape)
print(labels[1].item())

plt.figure(figsize=(8,16))
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.show()
    
    
imshow(torchvision.utils.make_grid(images))

In [ ]:
# Plotting images 

plt.figure(figsize=(10,10))
plt.subplot(321)
for i in range(25):
    ax1 = plt.subplot(5,5,i+1)
    plt.imshow(images[i+1].reshape(28,28).detach().numpy(),cmap='gray')
    #plt.title(labels[i+1].item())
    plt.axis('off')
plt.show()

## Train LeNet

In [ ]:
# Defining a custom LeNet module

class LeNet(nn.Module):                         # Extending nn.Module class 
    def __init__(self):                         # Constructor 
        super(LeNet,self).__init__()            # Calls the constructor of nn.Module
        self.cnn_model = nn.Sequential(         # nn.Sequentila allows multiple layers to stack together
            nn.Conv2d(1,6,5),                   #(N,1,28,28) -> (N,6,24,24)
            nn.Tanh(),                      
            nn.AvgPool2d(2,stride=2),           #(N,6,24,24) -> (N,6,12,12)
            nn.Conv2d(6,16,5),                  #(N,6,12,12) -> (N,16,8,8)
            nn.Tanh(),
            nn.AvgPool2d(2,stride=2)            #(N,16,8,8) -> (N,16,4,4)
            )
        
        self.fc_model = nn.Sequential(          # Fully connected layer 
            nn.Linear(256,120),
            nn.Tanh(),
            nn.Linear(120,84),
            nn.Tanh(),
            nn.Linear(84,10)
        
            )
        
# It get a batch of data which have defined earlier 
        
    def forward(self,x):     
        #print(x.shape)
        x = self.cnn_model(x)       
        #print(x.shape)
        x = x.view(x.size(0),-1)    # Flatning the inputs from tensors to vectors 
        #print(x.shape)
        x = self.fc_model(x)        # Passing the conv layer to fully connected layer
        #print(x.shape)
        return x
    
net = LeNet()#.to(device)
net

In [ ]:
print(net.fc_model[4].weight.shape,net.fc_model[4].bias.shape)
#list(net.parameters())

In [ ]:
# Evaluate on the basics of accuracy

def evaluation(dataloader):
    total , correct = 0,0
    for data in dataloader:
        inputs , labels = data
        inputs , labels = inputs.to(device) , labels.to(device)
        output = net(inputs)            
        max_pred,pred = torch.max(output.data,dim=1)
        total +=labels.size(0)
        correct +=(pred == labels).sum().item()  
    return 100 * correct / total

In [ ]:
net = LeNet().to(device)           # Creating object for LeNet() model and passing it to GPU 

loss_fn = nn.CrossEntropyLoss()    # It takes the highest value which is the predictions and mark it as 1
                                   # And mark rest of the values as zeros. 
opt = optim.Adam(net.parameters()) # Using adam Optimization algorithm , we can also specify the hyperparameters .

In [ ]:
# Fit the model and perform the training process 

%time
def fit(max_epochs =16):
    
    loss_arr = []
    loss_epoch_arr = []
    
    for epoch in range(max_epochs):
        for i, data in enumerate(trainloader,0): # Iterating through the train loader 
            inputs,labels = data
            inputs,labels = inputs.to(device),labels.to(device)

            opt.zero_grad()     # Reset the gradient in every iteration

            outputs = net(inputs)
            loss = loss_fn(outputs,labels)   # Loss forward pass
            loss.backward()                  # Loss backaed pass
            opt.step()                       # Update all the parameters by the given learnig rule

            loss_arr.append(loss.item())
        loss_epoch_arr.append(loss.item())
        print('Val accuracy: %0.2f , Train accuracy : %0.2f'%(evaluation(valloader),evaluation(trainloader)))

    plt.plot(loss_epoch_arr)
    plt.show()
    
fit()

In [ ]:
# Here we are visualiizing the behaviour of image in the first layer () of LeNet Architecture 

net = net.to('cpu')                    # Taking the model back to the CPU 
out = net.cnn_model[0](images)
print('Image Shape :',out.shape)
image_id = 3

plt.figure(figsize=(6,6))
plt.subplot(321)
for i in range(6):
    ax1 = plt.subplot(3,2,i+1)
    plt.imshow(out[image_id,i,:,:].detach().numpy(),cmap='gray')
plt.show()

In [ ]:
img , label = testset[0]
plt.imshow(img[0],cmap='gray')
print('Shape :',img.shape)
print('Label :',label)
print(img.unsqueeze(0).shape)

# img.unsqueeze simply adds another dimension at the begining of the 1x28x28 
# tensor, making it a 1x1x28x28 tensor, which the model views as a batch containing a single image.

In [ ]:
# Function which loads the prediction of our model

def predict_image(img,model):
    xb = img.unsqueeze(0)
    yb = net(xb)
    _,pred = torch.max(yb,dim=1)
    return pred[0].item()

In [ ]:
img , label = testset[16]
plt.imshow(img[0],cmap='gray')
print('Label :',label,'Predicted',predict_image(img,net))

In [ ]:
# Function which loads the prediction of our model

def predict_image(img,model):              # Here we are giving an input  tensor
    xb = img.unsqueeze(0)                  # Adding another dimention because our model accepts a bath
    yb = net(xb)                           # Passing the bacth into the model
    _,pred = torch.max(yb,dim=1)           # Getting the prediction from output (_ : will give the max prob and prob : will give the index of highest prob)              
    return pred[0].item()

# View the actual label and prediction of our model

plt.figure(figsize=(12,12))
plt.subplot(321)
for i in range(25):
    ax1 = plt.subplot(5,5,i+1)
    img , label = testset[i+1]
    plt.imshow(img[0],cmap='gray')
    if predict_image(img,net) == label:
        color = 'green'
    else:
        color = 'red'
    
    
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout(pad=2)
    plt.title('Pred:{},True:{}'.format(predict_image(img,net),label),color=color)
plt.show()

In [ ]:
torch.save(net.state_dict(), 'mnist-ResNet.pth')